In [1]:
# Thư viện
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
#read file 
df = pd.read_csv('CSGO_Market.csv')

In [20]:
Item_extra_list = ['StatTrak™', 'Souvenir', 'Normal']

Item_type_list = ['Rifle', 'Sniper Rifle', 'Pistol', 'SMG', 'Shotgun', 'Machinegun', 
                  'Knife', 'Gloves', 'Sticker', 'Container', 'Others']

Exterior_list = ['Battle-Scarred', 'Well-Worn', 'Field-Tested', 'Minimal Wear', 'Factory New', 'None']

Rarity_list = ['Base Grade', 'Consumer Grade', 'Industrial Grade', 'Mil-Spec', 'Restricted', 
               'Classified', 'Covert', 'Extraordinary',
                  'High Grade', 'Remarkable', 'Exotic', 'None']

In [5]:
df['Days to reach min'].mean()

562.8599895670318

In [9]:
print(len(df[df['Days to reach min'] < df['Days to reach max']]) / len(df))
print(len(df[df['Days to reach min'] > df['Days to reach max']]) / len(df))

0.5250912884715702
0.4748565466875326


In [31]:
print('Avg price by extra: ', '\n') 
for item in Item_extra_list:   
    print(item, ":", df[df['Extra'] == item]['Average price (VND)'].mean())
print(' ')
print('Avg price by type: ', '\n') 
for item in Item_type_list:   
    print(item, ":", df[df['Type'] == item]['Average price (VND)'].mean())
print(' ')

Avg price by extra:  

StatTrak™ : 1825015.5164573377
Souvenir : 638537.1542428073
Normal : 838875.8390217768
 
Avg price by type:  

Rifle : 884750.7902816571
Sniper Rifle : nan
Pistol : 291945.8927582509
SMG : 175595.33843803697
Shotgun : 113358.55286458871
Machinegun : 185540.11197481368
Knife : 5020627.454104856
Gloves : 8700526.042972788
Sticker : 137440.26580947376
Container : 250791.2563814286
Others : 31547.88150040375
 


In [32]:
print('Avg price by exterior: ', '\n') 
for item in Exterior_list:   
    print(item, ":", df[df['Exterior'] == item]['Average price (VND)'].mean())
print(' ')
print('Avg price by rarity: ', '\n') 
for item in Rarity_list:   
    print(item, ":", df[df['Rarity'] == item]['Average price (VND)'].mean())
print(' ')

Avg price by exterior:  

Battle-Scarred : 1157932.491807857
Well-Worn : 1108583.9248148182
Field-Tested : 1445926.5420879894
Minimal Wear : 1995114.501870035
Factory New : 2302772.4084820277
None : 142720.7978138154
 
Avg price by rarity:  

Base Grade : 55111.82257299852
Consumer Grade : 23959.061648847077
Industrial Grade : 86443.93714087906
Mil-Spec : 74667.4005763578
Restricted : 229928.84888554964
Classified : 916144.8561787975
Covert : 4537392.339362665
Extraordinary : 2645887.5469620214
High Grade : 39044.76628807081
Remarkable : 82370.4335950842
Exotic : 138678.88229737477
None : 163409.05964000005
 


- NO LINEAR REGRESSION for item of 'others' type since we do not care about them.

- NO LINEAR REGRESSION for item of 'container' type since we do not care about them.

separate linear regression model for each of gloves, knife and stickers as their rarity/extra/exterior works differently, other types get a shared model. 